In [ ]:
import nltk
# import all the resources for Natural Language Processing with Python
nltk.download("book")

from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re


In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Reading the data
path = "/content/drive/MyDrive/Saddam Sir/Thesis/Dataset/Suicide_twitter_data_final.xlsx"
path1 = "/content/drive/MyDrive/Saddam Sir/Thesis/Dataset/Reddit-Suicide-Data-Final.xlsx"
path2 = "/content/drive/MyDrive/Saddam Sir/Thesis/Dataset/Facebook-Suicide-Data-2.xlsx"

df = pd.read_excel(path)
df1 = pd.read_excel(path1)
df2 = pd.read_excel(path2)

# frames = [df, df1, df2]

# df3 = pd.concat(frames)
# df3.shape
# df3["text"][100]
df

In [5]:
df1['text'][180]

'  I am tired of being weak and destroyed.'

In [ ]:
df2

In [7]:
df3 = df

for ind in df1.index:
    # print(df['Name'][ind], df['Stream'][ind])
    # print(df1['text'][ind], df1['sentiment'][ind])

    df3 = df3.append({'text': df1['text'][ind], 'sentiment': df1['sentiment'][ind]}, ignore_index=True)

for ind in df2.index:
    # print(df['Name'][ind], df['Stream'][ind])
    # print(df2['text'][ind], df2['sentiment'][ind])

    df3 = df3.append({'text': df2['text'][ind], 'sentiment': df2['sentiment'][ind]}, ignore_index=True)

df1.shape[0]
df3.shape

(529, 2)

In [8]:
df3['text'][418]

'  Words on the street that world war 3 is getting started in order to exterminate all humans : Information is power, but like all power, there are those that want to keep it for themselves'

In [9]:
def preprocess(paragraph):
    text = re.sub(r'\[[0-9]*\]',' ',paragraph)
    text = re.sub(r'\s+',' ',text)
    text = text.lower()
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text

In [10]:
def sent_tok(paragraph):
  # sentences tokenize
  sentences = nltk.sent_tokenize(paragraph)
  # print(sentences)
  # tf_sen = sentences
  return sentences

In [11]:
df3['text'] = df3['text'].astype(str)
df3['text'][510]

'  World Suicide Prevention Day 2021 #SayNoToSuicide,'

In [12]:
df3['cleaned'] = df3['text'].apply(func = preprocess)
df3.head(4)
# display(df3)
# df3['text'][]

,text,sentiment,cleaned
0,b'@SmurphySuzanne @Sunderland_GM And more coun...,1.0,b'@smurphysuzanne @sunderland_gm and more coun...
1,b'RT @TatasenkoMana: This Method suicide 18 Da...,1.0,b'rt @tatasenkomana: this method suicide dazai...
2,b'RT @ennui365: Why in the name of Vince Foste...,1.0,"b'rt @ennui : why in the name of vince foster,..."
3,b'@Alan_G_Smith_BC @diana_murphy613 @JustinTru...,0.0,b'@alan_g_smith_bc @diana_murphy @justintrudea...


In [13]:
df3['cleaned'] = df3.cleaned.apply(func = sent_tok)
# df3['cleaned'][47]

In [14]:
df3['text'][420]

'  The Other Pandemic: Human Trafficking'

In [15]:
df3['cleaned'][442]

[' [event] mehndi\'s \\"heart of darkness\\" speech']

In [20]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [25]:
import gensim.models.keyedvectors as word2vec #need to use due to depreceated model
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve,  roc_auc_score, classification_report

import re
import logging

In [23]:
#and use regular expression to keep only letters we will use nltk Regular expression package
tkr = RegexpTokenizer('[a-zA-Z@]+')

text_split = []
cnt = 0
for x in df3['cleaned']:
    # print(x)

    text = str(x).lower().split()
    text = tkr.tokenize(str(text))
    # text_split.append(text)

    tfidf = TfidfVectorizer(stop_words='english')

    response = tfidf.fit_transform(x)

    # print (tfidf.get_feature_names_out())
    # Define the number of topics or components
    num_components=2

    # Create SVD object
    lsa = TruncatedSVD(n_components=num_components, n_iter=100, random_state=42)

    # Fit SVD model on data
    lsa.fit_transform(response)

    # Get Singular values and Components 
    Sigma = lsa.singular_values_ 
    V_transpose = lsa.components_.T

    # Print the topics with their terms
    terms = tfidf.get_feature_names_out()

    # print(cnt)
    cnt += 1
    temp = []
    for index, component in enumerate(lsa.components_):
        zipped = zip(terms, component)
        top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:3]
        top_terms_list=list(dict(top_terms_key).keys())
        # print("Topic "+str(index)+": ",top_terms_list)
        temp = temp + top_terms_list
      
    text_split.append(text+temp)

# print(cnt)
print(text_split[1:2])

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


[['b', 'rt', '@tatasenkomana', 'this', 'method', 'suicide', 'dazai', 'explained', 'to', 'oda', 'and', 'ango', 'letting', 'everyone', 'die', 'to', 'be', 'the', 'only', 'one', 'left', 'in', 'the', 'world', 'it', 'felt', 'xe', 'x', 'xa', 'ango', 'dazai', 'die']]


In [26]:
'''
Use pretrained Word2Vec model from google but trim the word list to 50,0000 compared to 300,000 in the original
Google pretrained model
'''

w2vModel = word2vec.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Pattern/Lab/Project/word2vec/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=50000)

In [28]:
#Convert words to integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_split)
X = tokenizer.texts_to_sequences(text_split)
# X[10]
# X.shape[1]
L = len(max(X, key=len))
L

182

In [29]:
#lenght of tweet to consider
maxlentweet = L
#add padding
X = pad_sequences(X, maxlen=maxlentweet)
print(X.shape)
X

(529, 182)


array([[   0,    0,    0, ...,  340, 1012,  141],
       [   0,    0,    0, ..., 1014, 1013,  464],
       [   0,    0,    0, ...,  394, 1016, 1803],
       ...,
       [   0,    0,    0, ...,    3, 1006,  193],
       [   0,    0,    0, ...,  298, 1008,  107],
       [   0,    0,    0, ...,   95,  328,  156]], dtype=int32)

In [30]:
w2vModel.syn0.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(50000, 300)

In [31]:
#create a embedding layer using Google pre triained word2vec (50000 words)
embedding_layer = Embedding(input_dim=w2vModel.syn0.shape[0], output_dim=w2vModel.syn0.shape[1], weights=[w2vModel.syn0], 
                            input_length=X.shape[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [32]:
#create model

lstm_out = 80

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(units=lstm_out))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 182, 300)          15000000  
                                                                 
 lstm (LSTM)                 (None, 80)                121920    
                                                                 
 dense (Dense)               (None, 1)                 81        
                                                                 
Total params: 15,122,001
Trainable params: 15,122,001
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
#split dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, df3['sentiment'], test_size= 0.1, random_state = 24)

In [34]:
#fit model
batch_size = 32
model.fit(X_train, Y_train, epochs=6, verbose=1, batch_size=batch_size)

Epoch 1/6
15/15 [==============================] - 11s 524ms/step - loss: 0.6746 - accuracy: 0.5567
Epoch 2/6
15/15 [==============================] - 8s 527ms/step - loss: 0.5231 - accuracy: 0.8214
Epoch 3/6
15/15 [==============================] - 6s 382ms/step - loss: 0.2882 - accuracy: 0.9097
Epoch 4/6
15/15 [==============================] - 7s 465ms/step - loss: 0.1734 - accuracy: 0.9475
Epoch 5/6
15/15 [==============================] - 7s 441ms/step - loss: 0.1346 - accuracy: 0.9727
Epoch 6/6
15/15 [==============================] - 6s 384ms/step - loss: 0.0811 - accuracy: 0.9748


In [35]:
#analyze the results
score, acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size=batch_size)
y_pred = model.predict(X_test)

2/2 - 1s - loss: 0.6417 - accuracy: 0.7170 - 669ms/epoch - 335ms/step
